<a href="https://colab.research.google.com/github/EliGorniak/KDD_Cup2009_MachineLearning_Kaggle_Project/blob/main/KDD_Cup_2009_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estudo de caso: KDD Cup 2009

A Administração de Relacionamento com o Consumidor é um elemento-chave das estratégias modernas de marketing. A maneira mais prática de adquirir conhecimento sobre consumidores é produzindo pontuações. Uma pontuação é a saída de um modelo e serve como avaliação de todas as instâncias de uma variável-alvo que se pretenda explicar. Ferramentas que produzem pontuações permitem projetar informações quantificáveis em uma dada população. A pontuação é calculada usando variáveis de entrada que descrevem as instâncias. As pontuações são então usadas pelo sistema de informação, por exemplo, para personalizar o relacionamento com o consumidor.

Em 2009, a conferência Knowledge Discovery and Data Mining (KDD) lançou um desafio envolvendo aprendizagem de máquina. Ela disponibilizou dados de consumidores da empresa de telecomunicações francesa Orange, que deveriam ser utilizados para a construção de um modelo capaz de prever três comportamentos do consumidor. A planilha fornecida tem 50000 instâncias e 230 variáveis, sendo as primeiras 190 variáveis numéricas e as demais 40 categóricas. As variáveis de saída do modelo eram o cancelamento da conta ('churn'), a tendência de usar novos produtos e serviços ('appetency') e a propensão para comprar upgrades ou adicionais com maior margem de lucro quando apresentados (upselling), fornecidos de forma binária (-1 para *não*, 1 para *sim*).

O trabalho apresentado aqui envolve a criação de um modelo para predição de *churn*, *appetency* e *upselling* com base nos dados fornecidos dos clientes da Orange. O critério para avaliação do modelo será a métrica AUC, que dimensiona o trade-off entre taxa de falsos positivos e taxa de verdadeiros positivos: quanto mais próximo de 1, menor o trade-off e melhor o modelo.

Detalhes sobre a competição: https://www.kdd.org/kdd-cup/view/kdd-cup-2009

# Etapa 1: Modelando 'churn' (cancelamento da conta)

### a. Importando as dependências

In [5]:
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier # algoritmos de classificação
from matplotlib import pyplot as plt # plotagem de gráficos
from sklearn.metrics import  roc_auc_score # AUC será a métrica de avaliação exigida na competição
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV # testar a performance dos modelos

### b. Carregando os dados

In [7]:
# definindo as variáveis que vão receber os dados
features = pd.read_csv('/orange_small_train.data', sep = '\t', na_filter = False) # sep é o separador, \t é uma tabulação, na_filter = False não vai remover os atributos com valores faltantes
outcome = pd.read_csv('/orange_small_train_churn.labels', header= None) # outcome são as respostas, header = None significa que não há um header para dar nome pras colunas

In [8]:
# verificando o tamanho do dataset
features.shape

(50000, 230)

In [19]:
features.head(10)

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,Var11,Var12,Var13,Var14,Var15,Var16,Var17,Var18,Var19,Var20,Var21,Var22,Var23,Var24,Var25,Var26,Var27,Var28,Var29,Var30,Var31,Var32,Var33,Var34,Var35,Var36,Var37,Var38,Var39,Var40,...,Var191,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,Var200,Var201,Var202,Var203,Var204,Var205,Var206,Var207,Var208,Var209,Var210,Var211,Var212,Var213,Var214,Var215,Var216,Var217,Var218,Var219,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
0,,,,,,1526,7,,,,,,184,,,,,,,,464,580,,14,128,,,166.56,,,,,,,0,,,3570,,,...,,bZkvyxLkBI,RO12,,taul,1K8T,lK27,ka_ns41,nQUveAzAF7,,,dXGu,9_Y1,FbIm,VpdQ,haYg,me75fM6ugJ,kIsH,,uKAI,L84s,XfqtO3UdzaXh_,,,,XTbPUYD,sH5Z,cJvF,FzaX,1YVfGrO,oslk,fXVEsaq,jySVZNlOJy,,,xb3V,RAYp,F2FyR07IdsN7I,,
1,,,,,,525,0,,,,,,0,,,,,,,,168,210,,2,24,,,353.52,,,,,,,0,,,4764966,,,...,,CEat0G8rTN,RO12,,taul,1K8T,2Ix5,qEdASpP,y2LIM01bE1,,,lg1t,9_Y1,k13i,sJzTlal,zm5i,me75fM6ugJ,kIsH,,uKAI,L84s,NhsEn4L,,,,kZJyVg2,,,FzaX,0AJo2f2,oslk,2Kb5FSF,LM8l689qOp,,,fKCe,RAYp,F2FyR07IdsN7I,,
2,,,,,,5236,7,,,,,,904,,,,,,,,1212,1515,,26,816,,,220.08,,,,,,,0,,,5883894,,,...,,eOQt0GoOh3,AERks4l,SEuy,taul,1K8T,ffXs,NldASpP,y4g9XoZ,vynJTq9,smXZ,4bTR,9_Y1,MGOA,VpdQ,haYg,DHn_WUyBhW_whjA88g9bvA64_,kIsH,,uKAI,L84s,UbxQ8lZ,,TTGHfSv,,pMWAe2U,bHR7,UYBR,FzaX,JFM1BiF,Al6ZaUT,NKv4yOc,jySVZNlOJy,,kG3k,Qu4f,02N6s8f,ib5G6X1eUxUn6,am7c,
3,,,,,,,0,,,,,,0,,,,,,,,,0,,,0,,,22.08,,,,,,,0,,,0,,,...,,jg69tYsGvO,RO12,,taul,1K8T,ssAy,_ybO0dd,4hMlgkf58mhwh,,,W8mQ,9_Y1,YULl,VpdQ,,me75fM6ugJ,kIsH,,uKAI,Mtgm,NhsEn4L,,,,kq0dQfu,eKej,UYBR,FzaX,L91KIiz,oslk,CE7uk3u,LM8l689qOp,,,FSa2,RAYp,F2FyR07IdsN7I,,
4,,,,,,1029,7,,,,,,3216,,,,,,,,64,80,,4,64,,,200,,,,,,,0,,,0,,,...,,IXSgUHShse,RO12,SEuy,taul,1K8T,uNkU,EKR938I,ThrHXVS,0v21jmy,smXZ,xklU,9_Y1,RVjC,sJzTlal,6JmL,me75fM6ugJ,kIsH,,uKAI,L84s,XfqtO3UdzaXh_,,SJs3duv,,11p4mKe,H3p7,UYBR,FzaX,OrnLfvc,oslk,1J2cvxe,LM8l689qOp,,kG3k,FSa2,RAYp,F2FyR07IdsN7I,mj86,
5,,,,,,658,7,,,,,,3156,,,,,,,,224,280,,2,72,,,200,,,,,,,5,,,0,,,...,,m9SrEy7Rm6,2Knk1KF,,taul,1K8T,lK27,fayYfhR,etM739XNb0Rf0,,,ae6C,9_Y1,yrDU,VpdQ,wMei,DHn_WUyBhW_whjA88g9bvA64_,kIsH,,uKAI,L84s,3vzwTT0wY25GE,,,,teAHwXo,ykzL,UYBR,FzaX,KbkKEj0,zCkv,QqVuch3,LM8l689qOp,,,Qcbd,02N6s8f,Zy3gnGM,am7c,
6,,,,,,1680,7,,,,,,2952,,,,,,,,308,385,,4,128,,,176.56,,,,,,,0,,,13158,,,...,,Qu0qrQKzJV,2Knk1KF,lvza,taul,1K8T,EJC9,ofiZR7x,LJF4fPp,QYxAlFM,smXZ,5Rb0,9_Y1,15m3,VpdQ,haYg,me75fM6ugJ,kIsH,,uKAI,Mtgm,XfqtO3UdzaXh_,,RQAGE01,,11p4mKe,NvHl,UYBR,FzaX,JO03372,oslk,XlgxB9z,LM8l689qOp,,kG3k,FSa2,RAYp,55YFVY9,am7c,
7,,,,,,77,0,,,,,,0,,,,,,,,32,40,,2,16,,,230.56,,,,,,,0,,,3776496,,,...,,eOQgUHShse,RO12,,LfvqpCtLOY,1K8T,Bxva,Xlthli9,GaiUdPAZp_,,,Mx5G,9_Y1,RVjC,VpdQ,IYzP,me75fM6ugJ,kIsH,,uKAI,Mtgm,NhsEn4L,,,,7WwuNea,9haV,UYBR,,U8IKsQe,oslk,R2LdzOv,,,,FSa2,RAYp,F2FyR07IdsN7I,,
8,,,,,,1176,7,,,,,,2912,,,,,,,,200,250,,2,64,,,300.32,,,,,,,0,,,6014460,,,...,,4e7Wq69R_D,RO12,SEuy,taul,1K8T,0Xwj,6CXYbuk,okUBQrgaYWgG0,z1Qe5zX,smXZ,1G9T,HLqf,Px52,VpdQ,IYzP,Kxdu,sBgB,,7A3j,L84s,9pUnzWLbztKTo,,yBN8Pcy,,kq0YABQ,,,FzaX,ROeipLp,zCkv,K2SqEo9,jySVZNlOJy,,kG3k,PM2D,6fzt,am14IcfM7tWLrUmRT52KtA,am7c,
9,,,,,,1141,7,,,,,,164,,,,,,,,208,260,,2,72,,,166.56,,,,,,,5,,,5317974,,,...,,J9Vr4RQZiT,2Knk1KF,SEuy,taul,1K8T,kNzO,jwhtMxl,koda1Jh,A4emZtf,smXZ,HJm0,9_Y1,mTeA,VpdQ,giwq,me75fM6ugJ,kIsH,,uKAI,L84s,h0lfDKh52u4GP,,IX6bJ7L,,IoI4mKe,b30L,UYBR,FzaX,fabLnWA,oslk,EPqQcw6,LM8l689qOp,,kG3k,FSa2,RAYp,55YFVY9,,


In [11]:
outcome.head()

,0
0,-1
1,1
2,-1
3,-1
4,-1


In [13]:
# verificando os valores únicos do dataset
np.unique(outcome)

array([-1,  1])

### c. Identificando as variáveis numéricas e categóricas

In [16]:
# variavel que vai armazenar o nome de todas as colunas (ou features)
all_vars = np.array(features.columns)
all_vars

array(['Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Var6', 'Var7', 'Var8',
       'Var9', 'Var10', 'Var11', 'Var12', 'Var13', 'Var14', 'Var15',
       'Var16', 'Var17', 'Var18', 'Var19', 'Var20', 'Var21', 'Var22',
       'Var23', 'Var24', 'Var25', 'Var26', 'Var27', 'Var28', 'Var29',
       'Var30', 'Var31', 'Var32', 'Var33', 'Var34', 'Var35', 'Var36',
       'Var37', 'Var38', 'Var39', 'Var40', 'Var41', 'Var42', 'Var43',
       'Var44', 'Var45', 'Var46', 'Var47', 'Var48', 'Var49', 'Var50',
       'Var51', 'Var52', 'Var53', 'Var54', 'Var55', 'Var56', 'Var57',
       'Var58', 'Var59', 'Var60', 'Var61', 'Var62', 'Var63', 'Var64',
       'Var65', 'Var66', 'Var67', 'Var68', 'Var69', 'Var70', 'Var71',
       'Var72', 'Var73', 'Var74', 'Var75', 'Var76', 'Var77', 'Var78',
       'Var79', 'Var80', 'Var81', 'Var82', 'Var83', 'Var84', 'Var85',
       'Var86', 'Var87', 'Var88', 'Var89', 'Var90', 'Var91', 'Var92',
       'Var93', 'Var94', 'Var95', 'Var96', 'Var97', 'Var98', 'Var99',
       'Var100', 'Va

In [17]:
# variavel que vai armazenar todas as primeiras 190 colunas do dataset que são apenas de variáveis numéricas
num_vars = np.array(all_vars[:190])
num_vars

array(['Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Var6', 'Var7', 'Var8',
       'Var9', 'Var10', 'Var11', 'Var12', 'Var13', 'Var14', 'Var15',
       'Var16', 'Var17', 'Var18', 'Var19', 'Var20', 'Var21', 'Var22',
       'Var23', 'Var24', 'Var25', 'Var26', 'Var27', 'Var28', 'Var29',
       'Var30', 'Var31', 'Var32', 'Var33', 'Var34', 'Var35', 'Var36',
       'Var37', 'Var38', 'Var39', 'Var40', 'Var41', 'Var42', 'Var43',
       'Var44', 'Var45', 'Var46', 'Var47', 'Var48', 'Var49', 'Var50',
       'Var51', 'Var52', 'Var53', 'Var54', 'Var55', 'Var56', 'Var57',
       'Var58', 'Var59', 'Var60', 'Var61', 'Var62', 'Var63', 'Var64',
       'Var65', 'Var66', 'Var67', 'Var68', 'Var69', 'Var70', 'Var71',
       'Var72', 'Var73', 'Var74', 'Var75', 'Var76', 'Var77', 'Var78',
       'Var79', 'Var80', 'Var81', 'Var82', 'Var83', 'Var84', 'Var85',
       'Var86', 'Var87', 'Var88', 'Var89', 'Var90', 'Var91', 'Var92',
       'Var93', 'Var94', 'Var95', 'Var96', 'Var97', 'Var98', 'Var99',
       'Var100', 'Va

In [18]:
# variavel que vai armazenar todas as ultimas colunas do dataset que são apenas de variáveis categóricas - as últimas 40 colunas
cat_vars = np.array(all_vars[190:])
cat_vars

array(['Var191', 'Var192', 'Var193', 'Var194', 'Var195', 'Var196',
       'Var197', 'Var198', 'Var199', 'Var200', 'Var201', 'Var202',
       'Var203', 'Var204', 'Var205', 'Var206', 'Var207', 'Var208',
       'Var209', 'Var210', 'Var211', 'Var212', 'Var213', 'Var214',
       'Var215', 'Var216', 'Var217', 'Var218', 'Var219', 'Var220',
       'Var221', 'Var222', 'Var223', 'Var224', 'Var225', 'Var226',
       'Var227', 'Var228', 'Var229', 'Var230'], dtype=object)

### d. Verificando a consistência das variáveis

*O objetivo é verificar o tipo de dado que cada variável possui, se numero, string, objeto, etc. Utilizando a biblioteca PANDAS e a função .dtypes.*

In [20]:
features.dtypes

Var1      object
Var2      object
Var3      object
Var4      object
Var5      object
           ...  
Var226    object
Var227    object
Var228    object
Var229    object
Var230    object
Length: 230, dtype: object

#### Pré-processamento dos dados para verificação do tipo de dados em cada variável




*Como o código anterior não detalhou o tipo de dados, precisamos fazer uma verificação mais passo a passo para confirmar se as variaveis numéricas são compostas apenas de números e se as variáveis categórias são compostas apenas por strings.*

*Começamos com a primeira coluna 'Var1':*

In [24]:
# agrupando os dados iguais e contando quantas vezes eles se repetem. Neste código, apenas para a primeira coluna.
var = features.groupby('Var1').size()
var

Var1
       49298
0        380
120        1
128        2
152        1
16        81
24        46
32        23
360        1
392        1
40        10
48         6
536        1
56         5
64         1
680        1
72         3
8        138
80         1
dtype: int64

*Como fica trabalhoso fazer a verificação para cada uma das colunas em separado, a linha de código abaixo fará a análise de forma otimizada:*

*Começamos trabalhando na variável que armazena todas as variaveis numérics, criando uma variável que vai gerar um dataframe pelo Pandas.*

*--> Començando com as variáveis NUMÉRICAS:*

In [46]:
counts_per_column = pd.DataFrame() # criando uma variável fazia no formato Dataframe
for col in num_vars: # vamos pegar cada coluna em num_vars
  col_count = features.groupby(col).size() # e corresponder em features, fazendo a contagem da quantidade de vezes que cada valor aparece em cada coluna
  counts_per_column = counts_per_column.append(col_count, ignore_index = True) # vamos povoar o dataframe a partir do .append

In [37]:
counts_per_column.shape # tamanho do dataframe - são 190 registros ou as colunas de num_vars e 351.788 valores

(190, 351788)

In [38]:
counts_per_column.head(10) # exibindo os 10 primeiros registros do dataframe

,,0,120,128,152,16,24,32,360,392,40,48,536,56,64,680,72,8,80,5,102,105,10617,108,1083,111,1113,114,1155,117,12,123,1242,126,12633,129,12927,130668,1332,1344,...,6651.18,67723.29,6893.82,69126.48,69166.71,693.45,693.99,7.47,7030.17,70445.25,7135.65,72177.84,73467.99,7584.66,7597.979,76999.23,77.67,78230.79,7922.7,79562.08,8022.96,81838.62,82165.95,8328.149,843.57,8573.939,8615.25,8660.25,86645.08,87838.47,8900.46,920.88,9336.06,9379.709,9441.36,9607.05,966.15,9772.021,98.1,9857.88
0,49298.0,380.0,1.0,2.0,1.0,81.0,46.0,23.0,1.0,1.0,10.0,6.0,1.0,5.0,1.0,1.0,3.0,138.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,48759.0,1240.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,48760.0,996.0,3.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,3.0,2.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,3.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,48421.0,1561.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,48513.0,898.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5529.0,976.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,132.0,NaN,NaN,NaN,153.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,160.0,NaN,NaN,NaN,NaN,147.0,NaN,109.0,NaN,NaN,NaN,NaN,130.0,NaN,NaN,NaN,NaN,NaN,81.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,5539.0,13090.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,50000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,49298.0,144.0,2.0,1.0,1.0,18.0,12.0,16.0,1.0,NaN,10.0,11.0,NaN,8.0,1.0,NaN,4.0,31.0,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,2.0,NaN,NaN,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,48513.0,902.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
counts_per_column.T.shape # .T vai fazer a matriz transposta, invertendo linhas e colunas, agora teremos 190 colunas e 351.788 valores

(351788, 190)

In [42]:
counts_per_column.T.head(10) # agora cada uma das linhas representa um valor e cada coluna representa uma das colunas do dataframe

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189
,49298.0,48759.0,48760.0,48421.0,48513.0,5529.0,5539.0,50000.0,49298.0,48513.0,48760.0,49442.0,5539.0,48760.0,50000.0,48513.0,48421.0,48421.0,48421.0,50000.0,5529.0,5009.0,48513.0,7230.0,5009.0,48513.0,48513.0,5011.0,49298.0,49298.0,50000.0,50000.0,49153.0,48759.0,5009.0,48759.0,48421.0,5009.0,50000.0,48759.0,...,49153.0,48421.0,5009.0,49298.0,48421.0,49306.0,48871.0,49127.0,48759.0,5009.0,48421.0,48759.0,5009.0,48421.0,49127.0,48513.0,50000.0,49298.0,50000.0,48759.0,48917.0,48513.0,5009.0,48421.0,50000.0,48760.0,48759.0,49354.0,48421.0,49298.0,5009.0,48421.0,48759.0,48759.0,50000.0,49298.0,49298.0,48759.0,28978.0,49667.0
0,380.0,1240.0,996.0,1561.0,898.0,976.0,13090.0,NaN,144.0,902.0,NaN,304.0,12258.0,1154.0,NaN,54.0,939.0,1200.0,1543.0,NaN,1815.0,2221.0,1052.0,16205.0,6629.0,1457.0,1467.0,388.0,694.0,195.0,NaN,NaN,523.0,1126.0,41219.0,692.0,964.0,9405.0,NaN,830.0,...,523.0,475.0,2221.0,273.0,1430.0,142.0,418.0,650.0,939.0,3848.0,1266.0,747.0,13457.0,1296.0,650.0,898.0,NaN,4.0,NaN,892.0,290.0,391.0,44857.0,964.0,NaN,1083.0,771.0,174.0,1289.0,144.0,41990.0,706.0,830.0,771.0,NaN,525.0,240.0,44.0,NaN,4.0
120,1.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,134.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,959.0,296.0,1.0,NaN,762.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,2.0,...,2.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,56.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.0,NaN
128,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,2.0,130.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1680.0,NaN,NaN,NaN,703.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,46.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,124.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,968.0,NaN,NaN,1.0,664.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,81.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,NaN,52.0,67.0,339.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,392.0,NaN,NaN,458.0,2903.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,NaN,NaN,NaN,NaN,NaN,90.0,...,55.0,NaN,NaN,NaN,NaN,NaN,66.0,NaN,NaN,1679.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.0,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.0,NaN,NaN,15.0,NaN,NaN,NaN
24,46.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,12.0,NaN,14.0,32.0,324.0,NaN,NaN,NaN,NaN,39.0,NaN,NaN,523.0,NaN,NaN,164.0,2627.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,53.0,...,44.0,56.0,NaN,NaN,NaN,NaN,62.0,1.0,NaN,1550.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,23.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,NaN,8.0,12.0,NaN,2.0,NaN
32,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN,3.0,20.0,237.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,477.0,NaN,NaN,72.0,2536.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,33.0,...,15.0,NaN,NaN,NaN,NaN,NaN,33.0,NaN,NaN,1041.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,NaN,NaN,3.0,NaN,NaN,NaN
360,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,71.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120.0,159.0,NaN,NaN,74.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,354.0,NaN
392,1.0,NaN,NaN,NaN,NaN,132.0,NaN,NaN,NaN,NaN,NaN,NaN,64.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.0,NaN,NaN,NaN,64.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,3.

*Agora vamos fazer uma codificação adicional, transformando o dataframe no layout definitivo transverso e verificando se todos os valores são realmente números:*

In [47]:
counts_per_column = counts_per_column.T # formato definitivo do dataframe
counts_per_column.index = counts_per_column.index.astype('str') # vai buscar apenas dados tipo string
counts_per_column.sort_index(inplace=True) # vai fazer a ordenação das variáveis a partir do index
print('Primeiras 10 categorias: ', counts_per_column.index[:10]) # mostrando os primeiros 
print('Últimas 10 categorias: ', counts_per_column.index[(len(counts_per_column.index)-10):])

Primeiras 10 categorias:  Index(['', '-10', '-1000060.0', '-10002.16', '-100034.8', '-1000420.0',
       '-1000468.0', '-1000496.0', '-1000516.0', '-1000548.0'],
      dtype='object')
Últimas 10 categorias:  Index(['9999040', '999920', '9999240', '99993', '9999400', '9999440',
       '9999480', '99995', '99996', '999971'],
      dtype='object')


*--> Fazendo o mesmo processo agora com as variáveis CATEGÓRICAS:*

In [48]:
counts_per_column = pd.DataFrame() # criando uma variável fazia no formato Dataframe
for col in cat_vars: # vamos pegar cada coluna em cat_vars
  col_count = features.groupby(col).size() # e corresponder em features, fazendo a contagem da quantidade de vezes que cada valor aparece em cada coluna
  counts_per_column = counts_per_column.append(col_count, ignore_index = True) # vamos povoar o dataframe a partir do .append

In [49]:
counts_per_column = counts_per_column.T # formato definitivo do dataframe
counts_per_column.index = counts_per_column.index.astype('str') # vai buscar apenas dados tipo string
counts_per_column.sort_index(inplace=True) # vai fazer a ordenação das variáveis a partir do index
print('Primeiras 10 categorias: ', counts_per_column.index[:10]) # mostrando os primeiros 
print('Últimas 10 categorias: ', counts_per_column.index[(len(counts_per_column.index)-10):])

Primeiras 10 categorias:  Index(['', '000J', '0062', '00AD', '00ARusu', '00AYONy', '00AhP4J', '00J8E9a',
       '00JHTSP', '00L3'],
      dtype='object')
Últimas 10 categorias:  Index(['zzQ9udm', 'zzQV', 'zzbGOGx', 'zzbGa54', 'zzcv6yg4s3UGv', 'zzfHvGh',
       'zzfLM_S', 'zzfUfAI', 'zzfo6HG', 'zzpZ'],
      dtype='object')


*Agora vamos fazer o tratamento dos VALORES FALTANTES, substituindo os espaços vazios por NaN*

In [50]:
# Utilizando a função .nan do Numpy, vamos substituir por NaN
features = features.replace('', np.nan)

In [51]:
# verificando se houve a substituição
features.head(10)

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,Var11,Var12,Var13,Var14,Var15,Var16,Var17,Var18,Var19,Var20,Var21,Var22,Var23,Var24,Var25,Var26,Var27,Var28,Var29,Var30,Var31,Var32,Var33,Var34,Var35,Var36,Var37,Var38,Var39,Var40,...,Var191,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,Var200,Var201,Var202,Var203,Var204,Var205,Var206,Var207,Var208,Var209,Var210,Var211,Var212,Var213,Var214,Var215,Var216,Var217,Var218,Var219,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
0,NaN,NaN,NaN,NaN,NaN,1526,7,NaN,NaN,NaN,NaN,NaN,184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,464,580,NaN,14,128,NaN,NaN,166.56,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,3570,NaN,NaN,...,NaN,bZkvyxLkBI,RO12,NaN,taul,1K8T,lK27,ka_ns41,nQUveAzAF7,NaN,NaN,dXGu,9_Y1,FbIm,VpdQ,haYg,me75fM6ugJ,kIsH,NaN,uKAI,L84s,XfqtO3UdzaXh_,NaN,NaN,NaN,XTbPUYD,sH5Z,cJvF,FzaX,1YVfGrO,oslk,fXVEsaq,jySVZNlOJy,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,525,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,168,210,NaN,2,24,NaN,NaN,353.52,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,4764966,NaN,NaN,...,NaN,CEat0G8rTN,RO12,NaN,taul,1K8T,2Ix5,qEdASpP,y2LIM01bE1,NaN,NaN,lg1t,9_Y1,k13i,sJzTlal,zm5i,me75fM6ugJ,kIsH,NaN,uKAI,L84s,NhsEn4L,NaN,NaN,NaN,kZJyVg2,NaN,NaN,FzaX,0AJo2f2,oslk,2Kb5FSF,LM8l689qOp,NaN,NaN,fKCe,RAYp,F2FyR07IdsN7I,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,5236,7,NaN,NaN,NaN,NaN,NaN,904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1212,1515,NaN,26,816,NaN,NaN,220.08,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,5883894,NaN,NaN,...,NaN,eOQt0GoOh3,AERks4l,SEuy,taul,1K8T,ffXs,NldASpP,y4g9XoZ,vynJTq9,smXZ,4bTR,9_Y1,MGOA,VpdQ,haYg,DHn_WUyBhW_whjA88g9bvA64_,kIsH,NaN,uKAI,L84s,UbxQ8lZ,NaN,TTGHfSv,NaN,pMWAe2U,bHR7,UYBR,FzaX,JFM1BiF,Al6ZaUT,NKv4yOc,jySVZNlOJy,NaN,kG3k,Qu4f,02N6s8f,ib5G6X1eUxUn6,am7c,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,NaN,22.08,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,NaN,...,NaN,jg69tYsGvO,RO12,NaN,taul,1K8T,ssAy,_ybO0dd,4hMlgkf58mhwh,NaN,NaN,W8mQ,9_Y1,YULl,VpdQ,NaN,me75fM6ugJ,kIsH,NaN,uKAI,Mtgm,NhsEn4L,NaN,NaN,NaN,kq0dQfu,eKej,UYBR,FzaX,L91KIiz,oslk,CE7uk3u,LM8l689qOp,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,1029,7,NaN,NaN,NaN,NaN,NaN,3216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64,80,NaN,4,64,NaN,NaN,200,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,NaN,...,NaN,IXSgUHShse,RO12,SEuy,taul,1K8T,uNkU,EKR938I,ThrHXVS,0v21jmy,smXZ,xklU,9_Y1,RVjC,sJzTlal,6JmL,me75fM6ugJ,kIsH,NaN,uKAI,L84s,XfqtO3UdzaXh_,NaN,SJs3duv,NaN,11p4mKe,H3p7,UYBR,FzaX,OrnLfvc,oslk,1J2cvxe,LM8l689qOp,NaN,kG3k,FSa2,RAYp,F2FyR07IdsN7I,mj86,NaN
5,NaN,NaN,NaN,NaN,NaN,658,7,NaN,NaN,NaN,NaN,NaN,3156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224,280,NaN,2,72,NaN,NaN,200,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,0,NaN,NaN,...,NaN,m9SrEy7Rm6,2Knk1KF,NaN,taul,1K8T,lK27,fayYfhR,etM739XNb0Rf0,NaN,NaN,ae6C,9_Y1,yrDU,VpdQ,wMei,DHn_WUyBhW_whjA88g9bvA64_,kIsH,NaN,uKAI,L84s,3vzwTT0wY25GE,NaN,NaN,NaN,teAHwXo,ykzL,UYBR,FzaX,KbkKEj0,zCkv,QqVuch3,LM8l689qOp,NaN,NaN,Qcbd,02N6s8f,Zy3gnGM,am7c,NaN
6,NaN,NaN,NaN,NaN,NaN,1680,7,NaN,NaN,NaN,NaN,NaN,2952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,308,385,NaN,4,128,NaN,NaN,176.56,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,13158,NaN,NaN,...,NaN,Qu0qrQKzJV,2Knk1KF,lvza,taul,1K8T,EJC9,ofiZR7x,LJF4fPp,QYxAlFM,smXZ,5Rb0,9_Y1,15m3,VpdQ,haYg,me75fM6ugJ,kIsH,NaN,uKAI,Mtgm,XfqtO3UdzaXh_,NaN,RQAGE01,NaN,11p4mKe,NvHl,UYBR,FzaX,JO03372,oslk,XlgxB9z,LM8l689qOp,NaN,kG3k,FSa2,RAYp,55YFVY9,am7c,NaN
7,NaN,NaN,NaN,NaN,NaN,77,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32,40,NaN,2,16,NaN,NaN,230.56,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,3776496,NaN,NaN,...,NaN,eOQgUHShse,RO12,NaN,LfvqpCtLOY,1K8T,Bxva,Xlthli9,GaiUdPAZp_,NaN,NaN,Mx5G,9_Y1,RVjC,VpdQ,IYzP,me75fM6ugJ,kIsH,NaN,uKAI,Mtgm,NhsEn4L,NaN,NaN,NaN,7WwuNea,9haV,UYBR,NaN,U8IKsQe,oslk,R2LdzOv,NaN,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,1176,7,NaN,NaN,NaN,NaN,NaN,2912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,250,NaN,2,64,NaN,NaN,300.32,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,6014460,NaN,NaN,...,NaN,4e7Wq69R_D,RO12,SEuy,taul,1K8

### e. Seleção de variáveis

#### Variáveis com muitos valores faltantes




#### Variáveis categóricas com muitas categorias




#### Seleção de variáveis baseaa em árvores de decisão

### f. Dividindo os dados em dataset para treino e dataset para teste

#### Modelagem preliminar




### d. Verificando a consistência das variáveis